In [250]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import logging
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [70]:
log = logging.getLogger("test_logger")

In [210]:
class RNN(nn.Module):
    """ 
    The (one to one) RNN class.
    """
    def __init__(self, input_size, hidden_size, output_size,bias = False):
        """ 
        Constructor of the class.
        Parameters
        ----------
        input_size: embedding size of the input
        hidden_size: hidden size
        output_size: embedding size of the output
        bias: set to "True" to enable bias
        Returns
        -------
        Nothing.
        """
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.aa = nn.Linear(hidden_size, hidden_size)
        self.ax = nn.Linear(input_size, hidden_size)
        self.ya = nn.Linear(hidden_size, output_size)
        self.a = torch.zeros(1, hidden_size)
        self.bias = bias
        if bias:
            self.ba = torch.rand(1,hidden_size)
            self.by = torch.rand(1, output_size)
        
    
    def forward(self, x):
        """ 
        Forward function of the class.
        Parameters
        ----------
        x: The input
        Returns
        -------
        y_t: The result of RNN.
        """
        if x.shape[-1] != self.input_size:
            log.error("Wrong input size!")
            return
        if not self.bias:
            a_t = F.sigmoid(self.aa(self.a) + self.ax(x))
            self.a = a_t
            y_t = F.sigmoid(self.ya(a_t))
        else:
            a_t = F.sigmoid(self.aa(self.axa) + self.ax(x)+self.ba)
            self.a = a_t
            y_t = F.sigmoid(self.ya(a_t) + self.by)
        return y_t
    
    def reset_a(self):
        self.a = torch.zeros(1, self.hidden_size)


In [184]:
test= RNN(32, 512, 256)
test.forward(torch.rand(64,32))

tensor([[0.4528, 0.4042, 0.5158,  ..., 0.5466, 0.4167, 0.6058],
        [0.4499, 0.3979, 0.5229,  ..., 0.5444, 0.4058, 0.6229],
        [0.4647, 0.4100, 0.5055,  ..., 0.5626, 0.4101, 0.6030],
        ...,
        [0.4450, 0.3999, 0.5147,  ..., 0.5493, 0.4056, 0.6146],
        [0.4523, 0.4012, 0.5130,  ..., 0.5513, 0.4066, 0.6111],
        [0.4618, 0.4145, 0.5125,  ..., 0.5512, 0.4138, 0.6081]],
       grad_fn=<SigmoidBackward0>)

In [27]:
testt= torch.rand(64,128,32)

In [28]:
testt[:,:,0].shape

torch.Size([64, 128])

In [241]:
class manyToOneRNN(nn.Module):
    """
    Many to one RNN class.
    """
    def __init__(self, input_times, input_size, hidden_size, output_size, bias = False):
        """
        Constructor of the class.
        Parameters
        ----------
        input_times: Times of inputs
        other parameters: Same with RNN class
        Returns
        -------
        Nothing
        """
        super(manyToOneRNN,self).__init__()
        self.rnn = RNN(input_size, hidden_size, output_size, bias)
        self.input_times = input_times
    def forward(self, x):
        """
        Forward function of this class
        Parameters
        ----------
        x: Input
        Returns
        -------
        Nothing
        """
        if x.shape[-1] != self.input_times:
            log.error("Wrong input size!")
            return
        for i in range(x.shape[-1]):
            y_t = self.rnn.forward(x[:,:,i])
        return y_t
    
    def reset_a(self):
        self.rnn.reset_a()

In [242]:
x_test = torch.reshape(torch.Tensor([1.,2.,3.,4.]),(1,1,4))
y_test = torch.reshape(torch.Tensor([1.]), (1,1))

In [243]:
y_test.shape

torch.Size([1, 1])

In [244]:
model = manyToOneRNN(4,1,4,1)

In [245]:
model.parameters()

<generator object Module.parameters at 0x7f2ae0505460>

In [246]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)


In [247]:
model.train()

manyToOneRNN(
  (rnn): RNN(
    (aa): Linear(in_features=4, out_features=4, bias=True)
    (ax): Linear(in_features=1, out_features=4, bias=True)
    (ya): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [248]:
for i in range(10):
#     model.hidden = model.init_hidden()
    
    model.reset_a()
    
    y_pred = model.forward(x_test)
    print(y_pred)
    optimizer.zero_grad()
    loss = loss_fn(y_pred, y_test)

    
    loss.backward()
    optimizer.step()


tensor([[0.5116]], grad_fn=<SigmoidBackward0>)
tensor([[0.6084]], grad_fn=<SigmoidBackward0>)
tensor([[0.7196]], grad_fn=<SigmoidBackward0>)
tensor([[0.8217]], grad_fn=<SigmoidBackward0>)
tensor([[0.8939]], grad_fn=<SigmoidBackward0>)
tensor([[0.9364]], grad_fn=<SigmoidBackward0>)
tensor([[0.9596]], grad_fn=<SigmoidBackward0>)
tensor([[0.9725]], grad_fn=<SigmoidBackward0>)
tensor([[0.9801]], grad_fn=<SigmoidBackward0>)
tensor([[0.9850]], grad_fn=<SigmoidBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [147]:
y_pred = model.forward(x_test)
y_pred

tensor([[0.4421]], grad_fn=<SigmoidBackward0>)

# View dataset

In [3]:
df = pd.read_csv("/kaggle/input/stock-exchange-data/indexData.csv")

In [55]:
df["Year"]= pd.to_datetime(df['Date']).dt.year
df["Month"] = pd.to_datetime(df['Date']).dt.month
df["Day"] = pd.to_datetime(df['Date']).dt.day

In [62]:
df.iloc[:,2:11]

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,528.690002,528.690002,528.690002,528.690002,528.690002,0.0,1965,12,31
1,527.210022,527.210022,527.210022,527.210022,527.210022,0.0,1966,1,3
2,527.840027,527.840027,527.840027,527.840027,527.840027,0.0,1966,1,4
3,531.119995,531.119995,531.119995,531.119995,531.119995,0.0,1966,1,5
4,532.070007,532.070007,532.070007,532.070007,532.070007,0.0,1966,1,6
...,...,...,...,...,...,...,...,...,...
112452,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0,2021,5,27
112453,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0,2021,5,28
112454,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0,2021,5,31
112455,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0,2021,6,1


In [75]:
df = df.dropna()
df

,Index,Date,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0,1965,12,31
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0,1966,1,3
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0,1966,1,4
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0,1966,1,5
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0,1966,1,6
...,...,...,...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0,2021,5,27
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0,2021,5,28
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0,2021,5,31
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0,2021,6,1


In [66]:
df_tensor = torch.tensor(df.iloc[:,2:11].drop(columns=["Close", "Adj Close"]).values)
df_tensor.shape

torch.Size([112457, 7])

In [68]:
target_tensor = torch.tensor(df['Close'].values)
target_tensor.shape

torch.Size([112457])

In [ ]:
class CustomStarDataset(Dataset)